In [1]:
!pip3 install pandas dash
!pip3 install wget

In [2]:
import wget
import pandas as pd

In [3]:
spacex_launch_csv =wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
pacex_df = pd.read_csv(spacex_launch_csv)
pacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [4]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv(spacex_launch_csv)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                html.Div([
                                    dcc.Dropdown(id='site-dropdown',
                                                 options=[
                                                     {'label': 'All Sites', 'value': 'ALL'},
                                                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                 ],
                                                 value='ALL',
                                                 placeholder="sites",
                                                 searchable=True
                                                ),
                                    html.Div([dcc.Graph(id='success-pie-chart')]),
                                    html.Div([html.H5('PayLoad range (kg)'),
                                              dcc.RangeSlider(id='payload-slider',
                                                    min=0, max=10000, step=1000,
                                                    marks={2500: '2500',
                                                           5000: '5000',
                                                           7500: '7500',
                                                           10000: '10000'},
                                                    value=[min_payload, max_payload])
                                             ]),
                                    html.Div([dcc.Graph(id='success-payload-scatter-chart')])
                                    # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                #html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                #html.Div(dcc.Graph(id='success-pie-chart')),
                                #html.Br(),

                                #html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                #html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])
                               ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df,
                     values='class', 
                     names='Launch Site', 
                     title='All Sites')
        return fig
    else:
        fd = spacex_df.loc[spacex_df['Launch Site'] == entered_site]
        fig=px.pie(fd,
                    names='class', 
                    title=entered_site)
        return fig
    
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])
def get_scatter_chart(entered_site, ps):
    #fd=spacex_df[(spacex_df['Payload Mass (kg)']) > ps[0]] and spacex_df[(spacex_df['Payload Mass (kg)'] < ps[1])]
    fd=spacex_df
    if entered_site == 'ALL':
        fig = px.scatter(fd[(fd['Payload Mass (kg)'] > ps[0]) &(fd['Payload Mass (kg)'] < ps[1]) ] ,
                        x='Payload Mass (kg)',
                        y='class',
                        color="Booster Version Category",
                        title='All sites')
        return fig
    else:
            fd = spacex_df.loc[spacex_df['Launch Site'] == entered_site]
            fig = px.scatter(fd[(fd['Payload Mass (kg)'] > ps[0]) &(fd['Payload Mass (kg)'] < ps[1]) ],
                        x='Payload Mass (kg)',
                        y='class',
                        color="Booster Version Category",
                        title=entered_site)
            return fig
    
    

# Run the app
if __name__ == '__main__':
    app.run_server()

<ipython-input-4-3b4ae1b20492>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-4-3b4ae1b20492>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Nov/2021 16:47:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 16:47:43] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 16:47:43] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 16:47:43] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 16:47:43] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 16:47:43] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 16:47:43] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 16:47:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 16:47:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 16:47:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 16:47:52] "POST /_dash-